In [15]:
import math
import argparse
import os
import PIL.Image
import shutil
import io
import sys
import time
import warnings
import xml.dom.minidom
import  tifffile
from collections import deque
from PIL import Image

NS_DEEPZOOM = 'http://schemas.microsoft.com/deepzoom/2008'

DEFAULT_RESIZE_FILTER = PIL.Image.ANTIALIAS
DEFAULT_IMAGE_FORMAT = 'jpg'

RESIZE_FILTERS = {
    'cubic': PIL.Image.CUBIC,
    'bilinear': PIL.Image.BILINEAR,
    'bicubic': PIL.Image.BICUBIC,
    'nearest': PIL.Image.NEAREST,
    'antialias': PIL.Image.ANTIALIAS,
    }

IMAGE_FORMATS = {
    'jpg': 'jpg',
    'png': 'png',
    }

In [23]:
class DeepZoomImageDescriptor(object):
    def __init__(self, width=None, height=None,
                 tile_size=254, tile_overlap=1, tile_format='jpg'):
        self.width = width
        self.height = height
        self.tile_size = tile_size
        self.tile_overlap = tile_overlap
        self.tile_format = tile_format
        self._num_levels = None

    def open(self, source):
        """Intialize descriptor from an existing descriptor file."""
        doc = xml.dom.minidom.parse(safe_open(source))
        image = doc.getElementsByTagName('Image')[0]
        size = doc.getElementsByTagName('Size')[0]
        self.width = int(size.getAttribute('Width'))
        self.height = int(size.getAttribute('Height'))
        self.tile_size = int(image.getAttribute('TileSize'))
        self.tile_overlap = int(image.getAttribute('Overlap'))
        self.tile_format = image.getAttribute('Format')

    def save(self, destination):
        """Save descriptor file."""
        with open(destination, 'w') as f:
            doc = xml.dom.minidom.Document()
            image = doc.createElementNS(NS_DEEPZOOM, 'Image')
            image.setAttribute('xmlns', NS_DEEPZOOM)
            image.setAttribute('TileSize', str(self.tile_size))
            image.setAttribute('Overlap', str(self.tile_overlap))
            image.setAttribute('Format', str(self.tile_format))
            size = doc.createElementNS(NS_DEEPZOOM, 'Size')
            size.setAttribute('Width', str(self.width))
            size.setAttribute('Height', str(self.height))
            image.appendChild(size)
            doc.appendChild(image)
            descriptor = doc.toxml()
            f.write(descriptor)

    @classmethod
    def remove(self, filename):
        """Remove descriptor file (DZI) and tiles folder."""
        _remove(filename)

    @property
    def num_levels(self):
        """Number of levels in the pyramid."""
        if self._num_levels is None:
            max_dimension = max(self.width, self.height)
            self._num_levels = int(math.ceil(math.log(max_dimension, 2))) + 1
        return self._num_levels

    def get_scale(self, level):
        """Scale of a pyramid level."""
        assert 0 <= level and level < self.num_levels, 'Invalid pyramid level'
        max_level = self.num_levels - 1
        return math.pow(0.5, max_level - level)

    def get_dimensions(self, level):
        """Dimensions of level (width, height)"""
        assert 0 <= level and level < self.num_levels, 'Invalid pyramid level'
        scale = self.get_scale(level)
        width = int(math.ceil(self.width * scale))
        height = int(math.ceil(self.height * scale))
        return (width, height)

    def get_num_tiles(self, level):
        """Number of tiles (columns, rows)"""
        assert 0 <= level and level < self.num_levels, 'Invalid pyramid level'
        w, h = self.get_dimensions( level )
        return (int(math.ceil(float(w) / self.tile_size)),
                int(math.ceil(float(h) / self.tile_size)))

    def get_tile_bounds(self, level, column, row):
        """Bounding box of the tile (x1, y1, x2, y2)"""
        assert 0 <= level and level < self.num_levels, 'Invalid pyramid level'
        offset_x = 0 if column == 0 else self.tile_overlap
        offset_y = 0 if row == 0 else self.tile_overlap
        x = (column * self.tile_size) - offset_x
        y = (row * self.tile_size) - offset_y
        level_width, level_height = self.get_dimensions(level)
        w = self.tile_size + (1 if column == 0 else 2) * self.tile_overlap
        h = self.tile_size + (1 if row == 0 else 2) * self.tile_overlap
        w = min(w, level_width  - x)
        h = min(h, level_height - y)
        return (x, y, x + w, y + h)

    
class ImageCreator(object):
    """Creates Deep Zoom images."""
    def __init__(self, tile_size=254, tile_overlap=1, tile_format='jpg',
                 image_quality=0.8, resize_filter=None, copy_metadata=False):
        self.tile_size = int(tile_size)
        self.tile_format = tile_format
        self.tile_overlap = _clamp(int(tile_overlap), 0, 10)
        self.image_quality = _clamp(image_quality, 0, 1.0)
        if not tile_format in IMAGE_FORMATS:
            self.tile_format = DEFAULT_IMAGE_FORMAT
        self.resize_filter = resize_filter
        self.copy_metadata = copy_metadata

    def get_image(self, level):
        """Returns the bitmap image at the given level."""
        assert 0 <= level and level < self.descriptor.num_levels, 'Invalid pyramid level'
        width, height = self.descriptor.get_dimensions(level)
        # don't transform to what we already have
        if self.descriptor.width == width and self.descriptor.height == height:
            return self.image
        if (self.resize_filter is None) or (self.resize_filter not in RESIZE_FILTERS):
            return self.image.resize((width, height), PIL.Image.ANTIALIAS)
        return self.image.resize((width, height), RESIZE_FILTERS[self.resize_filter])

    def tiles(self, level):
        """Iterator for all tiles in the given level. Returns (column, row) of a tile."""
        columns, rows = self.descriptor.get_num_tiles(level)
        for column in range(columns):
            for row in range(rows):
                yield (column, row)

    def create(self,image,source, destination):
        """Creates Deep Zoom image from source file and saves it to destination."""
        self.image = image
        width, height = self.image.size
        self.descriptor = DeepZoomImageDescriptor(width=width,
                                                  height=height,
                                                  tile_size=self.tile_size,
                                                  tile_overlap=self.tile_overlap,
                                                  tile_format=self.tile_format)
        # Create tiles
        image_files = _get_or_create_path(_get_files_path(destination))
        for level in range(self.descriptor.num_levels):
            level_dir = _get_or_create_path(os.path.join(image_files, str(level)))
            level_image = self.get_image(level)
            for (column, row) in self.tiles(level):
                bounds = self.descriptor.get_tile_bounds(level, column, row)
                tile = level_image.crop(bounds)
                format = self.descriptor.tile_format
                tile_path = os.path.join(level_dir,
                                         '{}_{}.{}'.format(column, row, format))
                with open(tile_path, 'wb') as tile_file:
                    if self.descriptor.tile_format == 'jpg':
                        jpeg_quality = int(self.image_quality * 100)
                        tile.save(tile_file, 'JPEG', quality=jpeg_quality)
                    else:
                        tile.save(tile_file, self.descriptor.tile_format.upper())
        # Create descriptor
        self.descriptor.save(destination)
def _clamp(val, min, max):
    if val < min:
        return min
    elif val > max:
        return max
    return val
def safe_open(path):
    with open(path, 'rb') as f:
        return io.BytesIO(f.read())
def _get_or_create_path(path):
    if not os.path.exists(path):
        os.makedirs(path)
    return path
def _get_files_path(path):
    return os.path.splitext(path)[0] + '_files'

In [25]:
source = "../tot.png"
des = "../tot.dzi"
source_image = tifffile.imread(source)
img = Image.fromarray(source_image)
creator = ImageCreator(
                            tile_size=256,
                           tile_format="jpg",
                           image_quality=1,
                           )
creator.create(img,source, des)
